In [1]:
# Scientific coding

## Naive Approach

Time complexity: $O(2^n)$

In [3]:
def fib(n):
    '''
    Calculates the fibinaci number for n
    using recursion.
    
    Params:
    ------
    n: int
        Assumes >= 1
    
    Returns:
    -------
    int
    '''
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [12]:
def fib_iter(n):
    last = 0
    nxt = 1
    
    for i in range(1, n):
        last, nxt = nxt, last+nxt
    
    return nxt

In [5]:
%%time
MAX_N = 10

for i in range(1, MAX_N):
    print(f'{i}:\t{fib(i)}')
    
print('complete')

1:	1
2:	2
3:	3
4:	5
5:	8
6:	13
7:	21
8:	34
9:	55
complete
CPU times: user 3.94 ms, sys: 657 µs, total: 4.6 ms
Wall time: 2.89 ms


In [7]:
fib(30)

168 ms ± 663 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
fib_iter(50)

12586269025

In [ ]:
%%time
MAX_N = 10

for i in range(1, MAX_N):
    print(f'{i}:\t{fib(i)}')
    
print('complete')

## Memoization

In [57]:
def cached(func):
    '''
    Momoization Decorator.  
    Creates a cache (lookup) of the historical
    calls to @func.
    
    Params:
    ------
    func: object
        Python function to decorate
        
    Returns:
    -------
    function: cache decorator
    '''
    history = {}
    def cache_decorator(*args):
        try:
            return history[args]
        except KeyError:
            val = func(*args)
            history[args] = val
            return val
    return cache_decorator

In [5]:
@cached
def fib2(n):
    '''
    Calculates the fibinaci number for n
    using recursion.
    
    Params:
    ------
    n: int
        Assumes >= 1
    
    Returns:
    -------
    int
    '''
    if n < 2:
        return 1
    else:
        return fib2(n-1) + fib2(n-2)

In [6]:
%%time
MAX_N = 40

for i in range(1, MAX_N):
    print(f'{i}:\t{fib2(i)}')
    
print('complete')

1:	1
2:	2
3:	3
4:	5
5:	8
6:	13
7:	21
8:	34
9:	55
10:	89
11:	144
12:	233
13:	377
14:	610
15:	987
16:	1597
17:	2584
18:	4181
19:	6765
20:	10946
21:	17711
22:	28657
23:	46368
24:	75025
25:	121393
26:	196418
27:	317811
28:	514229
29:	832040
30:	1346269
31:	2178309
32:	3524578
33:	5702887
34:	9227465
35:	14930352
36:	24157817
37:	39088169
38:	63245986
39:	102334155
complete
CPU times: user 1.15 ms, sys: 945 µs, total: 2.09 ms
Wall time: 1.35 ms


In [7]:
from functools import lru_cache

----

## Example 2: Binary Knapsack Problem

The knapsack problem is a problem in combinatorial optimization: Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

There are several ways to solve the knapsack problem.  We will first look at a naive approach and then how we can improve on the functions design.

We will use a very simple knapsack problem so we don't have excessive run times!  Even though this is simple you will see that it is possible to **substantially** improve the design of the function with some basic ideas.

```python
values = (50,100,150,200)
weights = (8,16,32,40)
max_weight = 64
```

### Naive brute force enumeration

A naive approach to solving this problem is to enumerate all possible combinations of items to include in the knapsack.  We then pick the combination with the largest value that does not exceed our weight constraint. 

This is a **binary** problem: an item is either included (1) or not included (0). An easy way to get all combinations of 0, 1 in a tuple or list of length `len(values)` is to use `itertools.product`.  In this case we need to iteate over 16 combinations.

In [13]:
from itertools import product

In [21]:
combs = list(product([0, 1], repeat=4))
combs

[(0, 0, 0, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 1, 1, 0),
 (0, 1, 1, 1),
 (1, 0, 0, 0),
 (1, 0, 0, 1),
 (1, 0, 1, 0),
 (1, 0, 1, 1),
 (1, 1, 0, 0),
 (1, 1, 0, 1),
 (1, 1, 1, 0),
 (1, 1, 1, 1)]

In [15]:
def knapsack_bruteforce(values, weights, max_weight):
    '''
    Enumerates all combinations of the knapsack packing options
    
    Params:
    -------
    values: float
        The value of the item
        
    weights: float
        the weight of the item
        
    max_weight: float
        the maximum allowable weight for the knapsack
        
    Returns:
    -------
    dict: binary tuple: (value, weight)
    '''
    results_val = []
    results_wt = []
    results = {}

    # all combinations of 0, 1 length 4
    combs = list(product([0, 1], repeat=4))
    
    # loop through the different combinations
    for option in combs:
        total_value = 0.0
        total_weight = 0.0
        
        # if the item is included then add in its weight, value
        for value, weight, include in zip(val, wt, option):
            if include == 1:
                total_value += value
                total_weight += weight
        results[option] = (total_value, total_weight)
        
    return results

In [16]:
val = (50, 100, 150, 200)
wt = (8, 16, 32, 40)
W = 64
knapsack_bruteforce(val, wt, W)

{(0, 0, 0, 0): (0.0, 0.0),
 (0, 0, 0, 1): (200.0, 40.0),
 (0, 0, 1, 0): (150.0, 32.0),
 (0, 0, 1, 1): (350.0, 72.0),
 (0, 1, 0, 0): (100.0, 16.0),
 (0, 1, 0, 1): (300.0, 56.0),
 (0, 1, 1, 0): (250.0, 48.0),
 (0, 1, 1, 1): (450.0, 88.0),
 (1, 0, 0, 0): (50.0, 8.0),
 (1, 0, 0, 1): (250.0, 48.0),
 (1, 0, 1, 0): (200.0, 40.0),
 (1, 0, 1, 1): (400.0, 80.0),
 (1, 1, 0, 0): (150.0, 24.0),
 (1, 1, 0, 1): (350.0, 64.0),
 (1, 1, 1, 0): (300.0, 56.0),
 (1, 1, 1, 1): (500.0, 96.0)}

Let's have a look at the execution speed

In [17]:
%timeit knapsack_bruteforce(val, wt, W)

10.6 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [32]:
def knapsack(n, weight_remaining):
    '''
    Find the optimal value of the knapsack without 
    exceeding the weight limit.
    
    Recursive method.
    Compares the value of the knapsack when the next 
    item is included versus when it is not included.
    
    Items are skipped if they exceed the weight limit.
    
    Params:
    ------
    n: int
        The current item 
    
    weight_remaining: int
        The remaining weight in the knapsack
        
    values: tuple
        The values of each item
        
    weights:
        The weights of each item

    Returns:
    --------
    int: the value of the knapsack
    '''
    if n < 0 or weight_remaining <= 0:
        # base condition - all weight used up
        return 0
    
    elif weights[n] > weight_remaining:
        # skip the item as exceeds weight
        return knapsack(n-1, weight_remaining)
    else:
        # best value when not including the item
        not_include = knapsack(n-1, weight_remaining)
        
        # best alue when item is included
        include = values[n] + knapsack(n-1, weight_remaining-weights[n])
        # return the best of the options.
        return max(not_include, include)

In [34]:
values = (50, 100, 150, 200)
weights = (8, 16, 32, 40)
max_weight = 64
knapsack(len(values)-1, max_weight)

350

In [ ]:
# execution speed of recursive procedure - should be faster...
%timeit knapsack(len(val)-1, W, val, wt)

In [58]:
@cached
def knapsack(n, weight_remaining):
    '''
    Find the optimal value of the knapsack without 
    exceeding the weight limit.
    
    Recursive method.
    Compares the value of the knapsack when the next 
    item is included versus when it is not included.
    
    Items are skipped if they exceed the weight limit.
    
    Params:
    ------
    n: int
        The current item 
    
    weight_remaining: int
        The remaining weight in the knapsack
        
    values: tuple
        The values of each item
        
    weights:
        The weights of each item

    Returns:
    --------
    int: the value of the knapsack
    '''
    # base condition 
    if n < 0 or weight_remaining <= 0:
        return 0
    elif weights[n] > weight_remaining:
        return knapsack(n-1, weight_remaining)
    else:
        # total when not including the item
        not_include = knapsack(n-1, weight_remaining)
        include = values[n] + knapsack(n-1, weight_remaining-weights[n])
        return max(not_include, include)

In [59]:
values = (50, 100, 150, 200)
weights = (8, 16, 32, 40)
max_weight = 64
knapsack(len(values)-1, max_weight)

350

Given five positive integers, find the minimum and maximum values that can be calculated by summing exactly four of the five integers. Then print the respective minimum and maximum values as a single line of two space-separated long integers.

Example
The minimum sum is and the maximum sum is . The function prints

In [60]:
arr = [1, 3, 5, 7, 9]

In [63]:
def smallest_n(arr, n=4):
    return sorted(arr)[:n]

def largest_n(arr, n=4):
    return sorted(arr, reverse=True)[:n]

In [64]:
print(smallest_n(arr))
print(largest_n(arr))

[1, 3, 5, 7]
[9, 7, 5, 3]
